# Import WOS & Scopus vers Infoscience : documentation des classes & fonctions + exemples

In [1]:
import pandas as pd
import os, logging
from dotenv import load_dotenv
import logging

In [2]:
# env var
load_dotenv(os.path.join(os.getcwd(), ".env"))

True

***

## Exemples d'utilisation des clients

In [3]:
from clients.wos_client_v2 import WosClient
from clients.scopus_client import ScopusClient

In [4]:
wos_epfl_query = "OG=(Ecole Polytechnique Federale de Lausanne)"
scopus_epfl_query = "AF-ID(60028186) AND (ORIG-LOAD-DATE AFT 20240722) AND (ORIG-LOAD-DATE BEF 20240831)" # pour Scopus le range de dates fait partie de la query
createdTimeSpan = "2024-01-01+2024-01-03" # Pour le WoS le range de date est un paramètre supplémentaire

### Nombre total de résultats

In [ ]:
## WoS
WosClient.count_results(usrQuery=wos_epfl_query)
WosClient.count_results(usrQuery=wos_epfl_query, createdTimeSpan=createdTimeSpan)

In [ ]:
## Scopus
ScopusClient.count_results(query=scopus_epfl_query)

### Récupération des IDs Scopus et WoS

Par défaut count = 10 

In [ ]:
## WoS
WosClient.fetch_ids(usrQuery=wos_epfl_query, count=50,createdTimeSpan=createdTimeSpan)

In [ ]:
## Scopus
ScopusClient.fetch_ids(query=scopus_epfl_query)

In [ ]:
## Loop
total = ScopusClient.count_results(query=scopus_epfl_query)
count = 5
ids = []
for i in range(1, int(total), int(count)):
    ids.extend(ScopusClient.fetch_ids(query = scopus_epfl_query, count = count, start =i))

### Extraction des métadonnées

4 formats de sortie possibles :

- "**digest**" (défault) :retourne les métadonnées
  - source
  - internal_id
  - doi
  - title
  - doctype
  - pubyear
- "**digest-ifs3**" : retourne les métadonnées du format **digest** plus
  - ifs3_doctype (nom de la collection Infoscience)
  - ifs3_collection_id (uuid de la collection Infoscience)
- "**ifs3**" : retourne les métadonnnées du format **digest-ifs3** plus
  - authors : liste d'objets auteur comprenant les métadonnées :
    - author (nom de l'auteur)
    - internal_author_id
    - orcid_id,
    - organizations
    - sub_organizations
- "**scopus**" ou "**wos**" : formats natifs du WoS ou Scopus


Par défault le format de sortie est "digest"

In [ ]:
# WoS format digest
WosClient.fetch_records(usrQuery=wos_epfl_query,count=2,createdTimeSpan=createdTimeSpan)

In [ ]:
# Scopus format ifs3
ScopusClient.fetch_records(format="ifs3",query=scopus_epfl_query,count=2)

In [ ]:
# Loop
total = ScopusClient.count_results(query=scopus_epfl_query)
count = 50
recs = []
for i in range(1, int(total), int(count)):
    recs.extend(ScopusClient.fetch_records(query = scopus_epfl_query, count = count, start =i))

***

## Exemples d'utilisation des harvesters

Les harvesters permettent d'unifier le processus de moissonnage multi-sources et de produire les dataframes des publications

In [ ]:
from data_pipeline.harvester import WosHarvester, ScopusHarvester

In [ ]:
default_queries = {
        "wos": "OG=(Ecole Polytechnique Federale de Lausanne)",
        "scopus": "AF-ID(60028186) OR AF-ID(60210159) OR AF-ID(60070536) OR AF-ID(60204330) OR AF-ID(60070531) OR AF-ID(60070534) OR AF-ID(60070538) OR AF-ID(60014951) OR AF-ID(60070529) OR AF-ID(60070532) OR AF-ID(60070535) OR AF-ID(60122563) OR AF-ID(60210160) OR AF-ID(60204331)",
        "openalex": "OPENALEX_QUERY_HERE",  # Placeholder for OpenAlex query in the future ?
        "zenodo": "ZENODO_QUERY_HERE"      # Placeholder for Zenodo query in teh future ?
    }
start_date = "2024-07-01"
end_date = "2024-07-10"

Par défaut le format de sortie est "ifs3"

In [ ]:
# Dataframe des publications WoS
wos_harvester = WosHarvester(start_date, end_date, default_queries["wos"])
wos_publications = wos_harvester.harvest()

In [ ]:
# Dataframe des publications Scopus
## format par défaut : ifs3
scopus_harvester = ScopusHarvester(start_date, end_date, default_queries["scopus"])
scopus_publications = scopus_harvester.harvest()

***

## Dédoublonnage

In [ ]:
from data_pipeline.deduplicator import DataFrameProcessor

In [ ]:
# Merge 
deduplicator = DataFrameProcessor(wos_publications, scopus_publications)
# Deduplicate the publications : first deduplicate operation between the sources
deduplicated_sources_df = deduplicator.deduplicate_dataframes()
# and second operation : filter by removing founded duplicates in Infoscience
df_final,df_unloaded = deduplicator.deduplicate_infoscience(deduplicated_sources_df)
# Generate main dataframes
df_metadata, df_authors = deduplicator.generate_main_dataframes(df_final)

***

## Consolidation données api.epfl.ch

In [3]:
from data_pipeline.enricher import Processor

In [4]:
df = pd.read_csv("harvested-data/2024_09_15/AddressesAndNames.csv", encoding="utf-8")
df.shape

(4618, 7)

### Etape 1 : Détection des auteurs affiliés à l'EPFL + nettoyage des noms auteurs + récupération des infos des unités via api.epfl.ch

In [5]:
# epfl_affiliation = True/False
processor = Processor(df)
df_epfl_affiliation_eval = processor.process(return_df=True)

In [6]:
# filtre sur les auteurs EPFL
df_tmp = df_epfl_affiliation_eval.copy()
df_tmp = df_tmp[df_tmp.epfl_affiliation]
df_tmp.shape

(236, 8)

#### a. Nettoyage des noms auteurs

In [7]:
#pd.set_option('display.max_rows', None)

In [8]:
# nouvelle colonne "author_cleaned"
processor = Processor(df_tmp)
df_cleaned_names = processor.clean_authors(return_df=True)

In [ ]:
df_cleaned_names

#### b. API EPFL infos

In [9]:
processor = Processor(df_cleaned_names[0:20])
df_api_epfl_infos = processor.api_epfl_reconciliation(return_df=True)

In [10]:
df_api_epfl_infos

,row_id,source,author,orcid_id,internal_author_id,organizations,suborganization,epfl_affiliation,author_cleaned,sciper_id_by_fullname,epfl_api_mainunit_id,epfl_api_mainunit_name
4,1,wos,"Zanoletti, Olivia",NaN,EGW-5587-2022,Swiss Fed Inst Technol Lausanne EPFL|Swiss Fed...,Behav Genet Lab,True,zanoletti olivia,196321,12544,COSEC-SV
5,1,wos,"Sandi, Carmen",0000-0001-7713-8321,DZQ-8659-2022,Swiss Fed Inst Technol Lausanne EPFL|Swiss Fed...,Behav Genet Lab,True,sandi carmen,160090,11365,EDNE-ENS
10,2,wos,"Adeli, Yeerlan",NaN,CBQ-4636-2022,Ecole Polytech Fed Lausanne|Swiss Federal Inst...,Inst Chem Sci & Engn|EPFL,True,adeli yeerlan,317625,None,None
11,2,wos,"Nuesch, Frank A.",NaN,IFL-4565-2023,Ecole Polytech Fed Lausanne|Swiss Federal Inst...,Inst Chem Sci & Engn|EPFL,True,nuesch frank a,0 résultat dans api.epfl.ch,None,None
15,3,wos,"Xia, Jianxing",NaN,GMI-0316-2022,Ecole Polytech Fed Lausanne EPFL|Swiss Federal...,Inst Chem Sci & Engn,True,xia jianxing,332654,9,LCOM
16,3,wos,"Nazeeruddin, Mohammad Khaja",0000-0003-3580-5960,B-1323-2008,Ecole Polytech Fed Lausanne EPFL|Swiss Federal...,Inst Chem Sci & Engn,True,nazeeruddin mohammad khaja,105958,12514,ISIC-DIV
28,4,scopus,Delalande A.,NaN,57219691961,60028186:École Polytechnique Fédérale de Lausanne,NaN,True,delalande a,Plus de 1 résultat dans api.epfl.ch,None,None
30,5,wos,"Kressner, Daniel",0000-0003-3369-2958,F-7600-2014,Ecole Polytech Fed Lausanne|Swiss Federal Inst...,Inst Math,True,kressner daniel,213191,12478,ANCHP
37,6,wos,"Termine, Alberto",NaN,GVV-6737-2022,Ecole Polytech Fed Lausanne EPFL|Swiss Federal...,Coll Humanities,True,termine alberto,392469,10475,CDH-DI
42,7,wos,"Scolaro, A.",0000-0001-9701-529X,DVU-2604-2022,Ecole Polytech Fed Lausanne|Swiss Federal Inst...,Lab Reactor Phys & Syst Behav LRS,True,scolaro a,Plus de 1 résultat dans api.epfl.ch,None,None


#### c. En mode pipeline

In [ ]:
processor = Processor(df)
df_api_epfl_infos = processor.process().clean_authors().api_epfl_reconciliation(return_df=True)

### Etape 2 : Complétion des métadonnées pour les auteurs non trouvés dans api.epfl.ch dans l'étape 1 : récupération des Orcid par un service Istex + récupération des métadonnées Orcid (orcid, firstname, lastname, vérif affiliation EPFL) + récupération des infos des unités via api.epfl.ch

In [ ]:
processor = Processor(df_api_epfl_infos)
df_epfl_enrich = processor.nameparse_authors().services_istex_orcid_reconciliation().orcid_data_reconciliation(return_df=True)

***

## Main

In [2]:
from data_pipeline.main import main